# Random Forest Model with natural data

## 1 Feature engineering

### 1.1 Importing libraries, transformers and path setting  

In [4]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [5]:
import os
cwd = os.getcwd()

counter = 0
print(counter, cwd)
while not cwd.endswith("drone_steering") and counter < 10:
    os.chdir('..')
    cwd = os.getcwd()
    counter = counter + 1
    print(counter, cwd)

0 /Users/lsafari/drone_steering/models
1 /Users/lsafari/drone_steering


### 1.2  Data loading

In [3]:
from app_local.module import DataEnsembler, GestureTransformer, DataResampler

de = DataEnsembler(120)
de.investigate_available_datafiles(data_dir='data/gesture/',is_frame_based=True)
de.combined_data_files_df


,filename_features,filename_labels
0,features_left_c_02_120.csv,labels_left_c_02.csv
1,features_left_l_01_120.csv,labels_left_l_01.csv
2,features_move_p_01_120.csv,labels_move_p_01.csv
3,features_move_c_01_120.csv,labels_move_c_01.csv
4,features_land_l_03_120.csv,labels_land_l_03.csv
5,features_move_l_02_120.csv,labels_move_l_02.csv
6,features_land_c_01_120.csv,labels_land_c_01.csv
7,features_land_l_02_120.csv,labels_land_l_02.csv
8,features_left_c_03_120.csv,labels_left_c_03.csv
9,features_right_c_02_120.csv,labels_right_c_02.csv


In [4]:
de.load_data()
de.pattern

'(?P<filename>(?P<filetype>[a-z]*)_(?P<movement>[a-z]*)_(?P<person>[a-z]*)_(?P<filenum>\\d*)(_(?P<frame_length>\\d*))?\\.csv)'

### 1.3 Data rescaling to actual video length

In [5]:
#de.rescale_data_frames(time_of_first_frame='avg',verbose=True)

### 1. 4 Generating training data

In [6]:
de.assemble_data(tolerance_range= 2)
de.display_information()

i: 0 	shape X: (550, 18, 16) 	shape y: (550,) 	count: 114
i: 1 	shape X: (541, 18, 16) 	shape y: (541,) 	count: 92
i: 2 	shape X: (590, 18, 16) 	shape y: (590,) 	count: 171
i: 3 	shape X: (616, 18, 16) 	shape y: (616,) 	count: 136
i: 4 	shape X: (466, 18, 16) 	shape y: (466,) 	count: 126
i: 5 	shape X: (470, 18, 16) 	shape y: (470,) 	count: 187
i: 6 	shape X: (440, 18, 16) 	shape y: (440,) 	count: 127
i: 7 	shape X: (436, 18, 16) 	shape y: (436,) 	count: 87
i: 8 	shape X: (549, 18, 16) 	shape y: (549,) 	count: 115
i: 9 	shape X: (923, 18, 16) 	shape y: (923,) 	count: 214
i: 10 	shape X: (545, 18, 16) 	shape y: (545,) 	count: 91
i: 11 	shape X: (476, 18, 16) 	shape y: (476,) 	count: 118
i: 12 	shape X: (621, 18, 16) 	shape y: (621,) 	count: 269
i: 13 	shape X: (551, 18, 16) 	shape y: (551,) 	count: 127
i: 14 	shape X: (505, 18, 16) 	shape y: (505,) 	count: 111
i: 15 	shape X: (535, 18, 16) 	shape y: (535,) 	count: 121
i: 16 	shape X: (514, 18, 16) 	shape y: (514,) 	count: 120
i: 17 	sha

### 1.5 Imbalance data treatment

In [7]:
dr = DataResampler()
dr.fit(de.X, de.y)
#dr.restrict_zero_labels(n=500, criterion=["variances_lowest"])
dr.display_information()

,abs_orig,pct_orig,abs_current,pct_current
0,14454,75.52,14454,75.52
1,701,3.66,701,3.66
2,763,3.99,763,3.99
3,991,5.18,991,5.18
4,882,4.61,882,4.61
5,660,3.45,660,3.45
6,689,3.6,689,3.6


In [8]:
dr.restrict_zero_labels(n=600,criterion="variances_lowest")

-------------------------------------
Current number of zero-labels: 14454
New number of zero-labels (selected with criterion="variances_lowest"): 600


(array([[[323.74369411, 205.4560586 , 308.99335097, ..., 148.67846194,
          271.25449345, 241.69583114],
         [324.55978596, 206.32017509, 308.1630041 , ..., 152.25815797,
          265.79939379, 240.52090492],
         [321.07795048, 198.56198579, 309.51981279, ..., 152.8917371 ,
          273.3064266 , 241.65127493],
         ...,
         [335.69136561, 184.02126529, 319.04156875, ..., 124.47094398,
          158.90030555, 162.97257529],
         [334.14316597, 181.55637775, 323.24348268, ..., 122.67106615,
          173.39366174, 180.37099933],
         [338.20502992, 187.33899018, 326.47399093, ..., 128.71612454,
          201.58922595, 214.39433576]],
 
        [[324.55978596, 206.32017509, 308.1630041 , ..., 152.25815797,
          265.79939379, 240.52090492],
         [321.07795048, 198.56198579, 309.51981279, ..., 152.8917371 ,
          273.3064266 , 241.65127493],
         [320.37548723, 200.74623447, 309.23026411, ..., 153.54252065,
          272.81516159, 242.0965

In [9]:
dr.display_information()

,abs_orig,pct_orig,abs_current,pct_current
0,14454,75.52,600,11.35
1,701,3.66,701,13.26
2,763,3.99,763,14.43
3,991,5.18,991,18.75
4,882,4.61,882,16.69
5,660,3.45,660,12.49
6,689,3.6,689,13.03


### 1.6 Data normalization

In [10]:
X = de.X.copy()
y = de.y.copy().astype("int32")
gt = GestureTransformer(byrow=True, feature_names= list(de.feature_names))
X = gt.transform(X, verbose = True)

---------------------------------------------------------
Performing row-based transformations:
X-Mean of Hip and Shoulders:
	Row-based Mean is taken over each sample and each timestep
	Shape: (19140, 18)
Y-Mean of Hip and Shoulders:
	Row-based Mean is taken over each sample and each timestep
	Shape: (19140, 18)
Scaling based on Hip-Shoulder distance:
	Row-based distance is calculated for each sample and each timestep
	Shape: (19140, 18)
-----------------------------------------------------
X-Shift:
            min   mean  median    max   std
timestep                                   
0         196.8  294.9   282.2  511.9  38.0
1         196.8  294.8   282.1  511.9  38.0
2         196.8  294.8   282.0  511.9  38.0
3         196.8  294.7   282.0  511.9  38.0
4         196.8  294.7   281.9  511.9  38.0
5         196.8  294.7   281.9  511.9  38.0
6         196.8  294.6   281.8  511.9  38.0
7         196.8  294.6   281.8  511.9  38.0
8         196.8  294.5   281.7  511.9  38.0
9         1

## 2 Model Training

### 2. 1 OneHot-Encoding of target vector

In [11]:
from sklearn.preprocessing import OneHotEncoder

In [12]:
ohe = OneHotEncoder(sparse = False)
print(type(ohe))
target = ohe.fit_transform(y.reshape(-1,1))
print("Before:", y.shape)
print("After:", target.shape)
target[0:5,:]

<class 'sklearn.preprocessing.data.OneHotEncoder'>
Before: (19140,)
After: (19140, 7)


array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.]])

### 2. 2 Train-Test-Split

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=10)
print("Training Data:", x_train.shape, y_train.shape)
print("Test Data:", x_test.shape, y_test.shape)

Training Data: (15312, 18, 16) (15312, 7)
Test Data: (3828, 18, 16) (3828, 7)


### 2. 3 Model architecture

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

### Train a simple Random Forest

In [15]:
model = RandomForestClassifier(
    n_estimators = 100, # 100
    criterion = "gini", # {gini, entropy}
    max_depth = None , # None
    n_jobs = -1,
    verbose = 1
)

model.fit(x_train.reshape(x_train.shape[0],-1),y_train)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    8.9s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

#### Training accuracy is already 100%

In [16]:
preds = model.predict(x_train.reshape(x_train.shape[0],-1))
(preds == y_train).all(axis = 1).sum()/y_train.shape[0]

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished


1.0

#### Test accuracy is 96 %

In [17]:
preds = model.predict(x_test.reshape(x_test.shape[0],-1))
(preds == y_test).all(axis = 1).sum()/y_test.shape[0]

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


0.9480146290491118

### 2. 4 Gridsearch with Crossvalidation

In [18]:
# Number of trees in random forest
n_estimators = [100, 500, 1000]
# Number of features to consider at every split
max_features = [5, 10, 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 110, num = 4)]
max_depth.append(None)

# Create the random grid
random_grid = {'randomforestclassifier__n_estimators': n_estimators,
               'randomforestclassifier__max_features': max_features,
               'randomforestclassifier__max_depth': max_depth,
              }

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
              }

print(random_grid)

{'n_estimators': [100, 500, 1000], 'max_features': [5, 10, 'sqrt'], 'max_depth': [5, 40, 75, 110, None]}


In [19]:
model.get_params().keys()

dict_keys(['bootstrap', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

#### Training takes quite a long time - do not needlessly repeat it

In [20]:
params = random_grid
scoring = "accuracy"
clf = GridSearchCV(model, params, n_jobs = -1, verbose = 10, scoring = scoring)
clf.fit(x_train.reshape(x_train.shape[0],-1),y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits
[CV] max_depth=5, max_features=5, n_estimators=100 ...................
[CV] max_depth=5, max_features=5, n_estimators=100 ...................
[CV] max_depth=5, max_features=5, n_estimators=100 ...................
[CV] max_depth=5, max_features=5, n_estimators=500 ...................
[CV] max_depth=5, max_features=5, n_estimators=500 ...................
[CV] max_depth=5, max_features=5, n_estimators=500 ...................
[CV] max_depth=5, max_features=5, n_estimators=1000 ..................
[CV] max_depth=5, max_features=5, n_estimators=1000 ..................


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.4s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.5s finished
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.4s finished
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Don

[CV]  max_depth=5, max_features=5, n_estimators=100, score=0.8516849529780565, total=   3.0s
[CV] max_depth=5, max_features=5, n_estimators=1000 ..................


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.5s finished


[CV]  max_depth=5, max_features=5, n_estimators=100, score=0.8491379310344828, total=   3.3s


...

[CV]  max_depth=None, max_features=sqrt, n_estimators=1000, score=0.9437695924764891, total= 6.0min


[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  3.7min finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    1.7s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    2.1s finished


[CV]  max_depth=None, max_features=sqrt, n_estimators=1000, score=0.9490595611285266, total= 3.9min


[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 44.6min finished
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   45.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   52.5s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [100, 500, 1000], 'max_features': [5, 10, 'sqrt'], 'max_depth': [5, 40, 75, 110, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=10)

In [21]:
clf.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_max_features', 'param_n_estimators', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'mean_train_score', 'std_train_score'])

In [22]:
clf.cv_results_["mean_test_score"]

array([0.85259927, 0.85031348, 0.848093  , 0.88100836, 0.88198798,
       0.88322884, 0.89276385, 0.89276385, 0.89269854, 0.93958986,
       0.94181034, 0.9422675 , 0.94272466, 0.94324713, 0.9438349 ,
       0.94403083, 0.94533699, 0.94520637, 0.94102665, 0.94272466,
       0.94200627, 0.94357367, 0.94435737, 0.94448798, 0.94350836,
       0.94592476, 0.94546761, 0.94122257, 0.94194096, 0.94233281,
       0.94318182, 0.94363898, 0.94376959, 0.94468391, 0.94546761,
       0.94553292, 0.94037356, 0.94135319, 0.94181034, 0.94357367,
       0.94403083, 0.94435737, 0.94337774, 0.94585946, 0.94533699])

#### Determine performance of best Gridsearch-Model on the Holdout-Set

In [23]:
preds = clf.best_estimator_.predict(x_test.reshape(x_test.shape[0],-1))
(preds == y_test).all(axis = 1).sum()/y_test.shape[0]

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.6s finished


0.9487983281086729

In [7]:
type(loaded_model.best_estimator_)

sklearn.ensemble.forest.RandomForestClassifier

In [8]:
from sklearn.externals import joblib
joblib.dump(loaded_model.best_estimator_,'models/model_gesture_rf_nip120_sa600_variances_lowest.pkl')

['models/model_gesture_rf_nip120_sa600_variances_lowest.pkl']

In [6]:
from sklearn.externals import joblib
loaded_model=joblib.load('models/model_gesture_rf_nip120_sa600_variances_lowest.pkl')
loaded_model

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [100, 500, 1000], 'max_features': [5, 10, 'sqrt'], 'max_depth': [5, 40, 75, 110, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=10)

In [3]:
os.getcwd()

NameError: name 'os' is not defined